# 第4章: 確率分布

この章では、統計学で重要な確率分布について学びます。

## 学習目標
- 確率の基本概念
- 離散確率分布（二項分布、ポアソン分布）
- 連続確率分布（正規分布、t分布、カイ二乗分布）
- 中心極限定理

In [ ]:
// ライブラリの読み込み
const math = await import('https://esm.sh/mathjs@13.0.0');
const ss = await import('https://esm.sh/simple-statistics@7.8.8');
console.log('Libraries loaded!');

## 4.1 確率の基本

### 4.1.1 基本概念

In [ ]:
// 確率の基本法則
console.log('=== 確率の基本法則 ===');
console.log('');
console.log('1. 0 ≤ P(A) ≤ 1');
console.log('2. P(全事象) = 1');
console.log('3. P(A∪B) = P(A) + P(B) - P(A∩B)');
console.log('');

// サイコロの例
console.log('【例: サイコロを1回振る】');
const diceOutcomes = [1, 2, 3, 4, 5, 6];
console.log('標本空間:', diceOutcomes);
console.log('各目が出る確率: 1/6 ≈', (1/6).toFixed(4));

// 偶数が出る確率
const evenNumbers = diceOutcomes.filter(x => x % 2 === 0);
console.log('偶数が出る確率:', evenNumbers.length + '/' + diceOutcomes.length, '=', (evenNumbers.length / diceOutcomes.length).toFixed(4));

// 4以上が出る確率
const fourOrMore = diceOutcomes.filter(x => x >= 4);
console.log('4以上が出る確率:', fourOrMore.length + '/' + diceOutcomes.length, '=', (fourOrMore.length / diceOutcomes.length).toFixed(4));

### 4.1.2 条件付き確率とベイズの定理

In [ ]:
// ベイズの定理の例：検査の精度
// 病気Aの有病率: 1%
// 検査の感度（病気の人を陽性と判定）: 99%
// 検査の特異度（健康な人を陰性と判定）: 95%

const prevalence = 0.01;      // 有病率 P(D)
const sensitivity = 0.99;     // 感度 P(+|D)
const specificity = 0.95;     // 特異度 P(-|¬D)
const falsePositive = 1 - specificity;  // 偽陽性率 P(+|¬D)

console.log('=== ベイズの定理の例 ===');
console.log('有病率 P(D):', prevalence);
console.log('感度 P(+|D):', sensitivity);
console.log('特異度 P(-|¬D):', specificity);
console.log('偽陽性率 P(+|¬D):', falsePositive);

// P(+) = P(+|D)P(D) + P(+|¬D)P(¬D)
const pPositive = sensitivity * prevalence + falsePositive * (1 - prevalence);
console.log('\n陽性になる確率 P(+):', pPositive.toFixed(4));

// ベイズの定理: P(D|+) = P(+|D)P(D) / P(+)
const pDiseaseGivenPositive = (sensitivity * prevalence) / pPositive;
console.log('\n陽性の場合に本当に病気である確率 P(D|+):', pDiseaseGivenPositive.toFixed(4));
console.log('→ 約', (pDiseaseGivenPositive * 100).toFixed(1) + '%');

console.log('\n注意: 有病率が低いと、陽性でも実際は病気でない可能性が高い！');

### 4.1.3 組み合わせと順列

In [ ]:
// 階乗
console.log('=== 階乗 ===');
for (let n = 0; n <= 10; n++) {
    console.log(n + '! =', math.factorial(n));
}

// 順列 P(n,r) = n! / (n-r)!
function permutation(n, r) {
    return math.factorial(n) / math.factorial(n - r);
}

console.log('\n=== 順列 P(n,r) ===');
console.log('P(5,3) = 5人から3人を選んで並べる:', permutation(5, 3));
console.log('P(10,2) = 10人から2人を選んで並べる:', permutation(10, 2));

// 組み合わせ C(n,r) = n! / (r!(n-r)!)
console.log('\n=== 組み合わせ C(n,r) ===');
console.log('C(5,3) = 5人から3人を選ぶ:', math.combinations(5, 3));
console.log('C(10,2) = 10人から2人を選ぶ:', math.combinations(10, 2));
console.log('C(52,5) = 52枚から5枚を選ぶ（ポーカー）:', math.combinations(52, 5));

## 4.2 離散確率分布

### 4.2.1 二項分布 (Binomial Distribution)

n回の独立なベルヌーイ試行で、成功確率pの試行がk回成功する確率

In [ ]:
// 二項分布の確率質量関数
// P(X=k) = C(n,k) * p^k * (1-p)^(n-k)
function binomialPMF(k, n, p) {
    return math.combinations(n, k) * Math.pow(p, k) * Math.pow(1 - p, n - k);
}

// 例: コインを10回投げて表が出る回数
const nTrials = 10;
const pSuccess = 0.5;

console.log('=== 二項分布 B(' + nTrials + ', ' + pSuccess + ') ===');
console.log('コインを10回投げて表が出る回数の確率分布');
console.log('');

let totalProb = 0;
for (let k = 0; k <= nTrials; k++) {
    const prob = binomialPMF(k, nTrials, pSuccess);
    totalProb += prob;
    const bar = '█'.repeat(Math.round(prob * 50));
    console.log(`P(X=${k.toString().padStart(2)}): ${prob.toFixed(4)} ${bar}`);
}
console.log('合計:', totalProb.toFixed(4));

// 期待値と分散
console.log('\n期待値 E[X] = np =', nTrials * pSuccess);
console.log('分散 Var[X] = np(1-p) =', nTrials * pSuccess * (1 - pSuccess));

In [ ]:
// 二項分布の累積分布関数
function binomialCDF(k, n, p) {
    let sum = 0;
    for (let i = 0; i <= k; i++) {
        sum += binomialPMF(i, n, p);
    }
    return sum;
}

// 例: 品質検査
// 不良率5%の製品を20個検査して、不良品が2個以下である確率
const nProducts = 20;
const defectRate = 0.05;

console.log('=== 品質検査の例 ===');
console.log('不良率:', defectRate * 100 + '%');
console.log('検査数:', nProducts);
console.log('');

console.log('不良品が0個の確率:', binomialPMF(0, nProducts, defectRate).toFixed(4));
console.log('不良品が1個の確率:', binomialPMF(1, nProducts, defectRate).toFixed(4));
console.log('不良品が2個の確率:', binomialPMF(2, nProducts, defectRate).toFixed(4));
console.log('');
console.log('不良品が2個以下の確率:', binomialCDF(2, nProducts, defectRate).toFixed(4));
console.log('不良品が3個以上の確率:', (1 - binomialCDF(2, nProducts, defectRate)).toFixed(4));

### 4.2.2 ポアソン分布 (Poisson Distribution)

一定時間・空間内に発生する稀な事象の回数の分布

In [ ]:
// ポアソン分布の確率質量関数
// P(X=k) = (λ^k * e^(-λ)) / k!
function poissonPMF(k, lambda) {
    return (Math.pow(lambda, k) * Math.exp(-lambda)) / math.factorial(k);
}

// 例: 1時間に平均3件の電話がかかってくる
const avgCalls = 3;

console.log('=== ポアソン分布 Po(' + avgCalls + ') ===');
console.log('1時間に平均3件の電話がかかってくる場合');
console.log('');

for (let k = 0; k <= 10; k++) {
    const prob = poissonPMF(k, avgCalls);
    const bar = '█'.repeat(Math.round(prob * 30));
    console.log(`P(X=${k.toString().padStart(2)}): ${prob.toFixed(4)} ${bar}`);
}

// 期待値と分散
console.log('\n期待値 E[X] = λ =', avgCalls);
console.log('分散 Var[X] = λ =', avgCalls);
console.log('（ポアソン分布では期待値と分散が等しい）');

In [ ]:
// ポアソン分布の累積確率
function poissonCDF(k, lambda) {
    let sum = 0;
    for (let i = 0; i <= k; i++) {
        sum += poissonPMF(i, lambda);
    }
    return sum;
}

// 実用例
console.log('=== ポアソン分布の活用例 ===');
console.log('');

// 例1: 交通事故
const accidentsPerDay = 2.5;  // 1日平均2.5件の事故
console.log('【交通事故】1日平均2.5件');
console.log('0件の確率:', poissonPMF(0, accidentsPerDay).toFixed(4));
console.log('5件以上の確率:', (1 - poissonCDF(4, accidentsPerDay)).toFixed(4));

console.log('');

// 例2: Webサイトアクセス
const visitsPerMinute = 10;  // 1分間に平均10アクセス
console.log('【Webアクセス】1分間に平均10アクセス');
console.log('15アクセス以上の確率:', (1 - poissonCDF(14, visitsPerMinute)).toFixed(4));

## 4.3 連続確率分布

### 4.3.1 正規分布（ガウス分布）

最も重要な連続確率分布。多くの自然現象がこの分布に従う。

In [ ]:
// 正規分布の確率密度関数
// f(x) = (1 / (σ√(2π))) * e^(-(x-μ)²/(2σ²))
function normalPDF(x, mu, sigma) {
    const coefficient = 1 / (sigma * Math.sqrt(2 * Math.PI));
    const exponent = -Math.pow(x - mu, 2) / (2 * Math.pow(sigma, 2));
    return coefficient * Math.exp(exponent);
}

// 標準正規分布 N(0, 1)
console.log('=== 標準正規分布 N(0, 1) ===');
console.log('');

// 確率密度関数の形状
for (let x = -3; x <= 3; x += 0.5) {
    const y = normalPDF(x, 0, 1);
    const bar = '█'.repeat(Math.round(y * 25));
    console.log(`f(${x.toFixed(1).padStart(4)}): ${y.toFixed(4)} ${bar}`);
}

In [ ]:
// 正規分布の累積分布関数（simple-statisticsを使用）
console.log('=== 標準正規分布の累積確率 ===');
console.log('');

// Φ(z) = P(Z ≤ z)
const zValues = [-3, -2, -1, 0, 1, 2, 3];

zValues.forEach(z => {
    const cdf = ss.cumulativeStdNormalProbability(z);
    console.log(`Φ(${z.toString().padStart(2)}) = P(Z ≤ ${z.toString().padStart(2)}) = ${cdf.toFixed(4)}`);
});

console.log('');
console.log('【重要な確率】');
console.log('P(-1 ≤ Z ≤ 1) = 68.27% (1σの範囲)');
console.log('P(-2 ≤ Z ≤ 2) = 95.45% (2σの範囲)');
console.log('P(-3 ≤ Z ≤ 3) = 99.73% (3σの範囲)');

// 実際に計算で確認
const within1sigma = ss.cumulativeStdNormalProbability(1) - ss.cumulativeStdNormalProbability(-1);
const within2sigma = ss.cumulativeStdNormalProbability(2) - ss.cumulativeStdNormalProbability(-2);
const within3sigma = ss.cumulativeStdNormalProbability(3) - ss.cumulativeStdNormalProbability(-3);

console.log('');
console.log('計算結果:');
console.log('1σ:', (within1sigma * 100).toFixed(2) + '%');
console.log('2σ:', (within2sigma * 100).toFixed(2) + '%');
console.log('3σ:', (within3sigma * 100).toFixed(2) + '%');

In [ ]:
// 正規分布の活用例
console.log('=== 正規分布の活用例 ===');
console.log('');

// 例: テストの点数が平均70点、標準偏差10点の正規分布に従う
const testMean = 70;
const testStd = 10;

console.log('【テストの点数】平均=' + testMean + '点, 標準偏差=' + testStd + '点');
console.log('');

// 80点以上を取る確率
const z80 = (80 - testMean) / testStd;
const pAbove80 = 1 - ss.cumulativeStdNormalProbability(z80);
console.log('80点以上の確率:');
console.log('  z = (80 - 70) / 10 =', z80);
console.log('  P(X ≥ 80) =', (pAbove80 * 100).toFixed(2) + '%');

console.log('');

// 60点から80点の間の確率
const z60 = (60 - testMean) / testStd;
const pBetween = ss.cumulativeStdNormalProbability(z80) - ss.cumulativeStdNormalProbability(z60);
console.log('60点〜80点の確率:');
console.log('  P(60 ≤ X ≤ 80) =', (pBetween * 100).toFixed(2) + '%');

console.log('');

// 上位10%のライン
const z90percentile = ss.probit(0.90);  // 上位10%点
const score90percentile = testMean + z90percentile * testStd;
console.log('上位10%のライン:');
console.log('  z₀.₉₀ =', z90percentile.toFixed(4));
console.log('  点数:', score90percentile.toFixed(1) + '点以上');

### 4.3.2 t分布（スチューデントのt分布）

小標本から母平均を推定する際に使用

In [ ]:
// t分布の確率密度関数
function tPDF(t, df) {
    const coefficient = math.gamma((df + 1) / 2) / (Math.sqrt(df * Math.PI) * math.gamma(df / 2));
    return coefficient * Math.pow(1 + (t * t) / df, -(df + 1) / 2);
}

console.log('=== t分布の形状 ===');
console.log('自由度による形状の変化');
console.log('');

// 異なる自由度でのt分布
const degreesOfFreedom = [1, 5, 30];

console.log('t = 0 での確率密度:');
degreesOfFreedom.forEach(df => {
    console.log(`  df=${df.toString().padStart(2)}: ${tPDF(0, df).toFixed(4)}`);
});
console.log(`  正規分布: ${normalPDF(0, 0, 1).toFixed(4)}`);

console.log('');
console.log('t = 2 での確率密度:');
degreesOfFreedom.forEach(df => {
    console.log(`  df=${df.toString().padStart(2)}: ${tPDF(2, df).toFixed(4)}`);
});
console.log(`  正規分布: ${normalPDF(2, 0, 1).toFixed(4)}`);

console.log('');
console.log('→ 自由度が大きくなるほど正規分布に近づく');

### 4.3.3 カイ二乗分布（χ²分布）

分散の推定や適合度検定に使用

In [ ]:
// カイ二乗分布の確率密度関数
function chiSquaredPDF(x, k) {
    if (x <= 0) return 0;
    const coefficient = 1 / (Math.pow(2, k/2) * math.gamma(k/2));
    return coefficient * Math.pow(x, k/2 - 1) * Math.exp(-x/2);
}

console.log('=== カイ二乗分布 χ²(k) ===');
console.log('自由度kによる形状の変化');
console.log('');

const kValues = [2, 5, 10];

kValues.forEach(k => {
    console.log(`χ²(${k})分布:`);
    console.log(`  期待値 E[X] = ${k}`);
    console.log(`  分散 Var[X] = ${2 * k}`);
    console.log(`  f(${k}) = ${chiSquaredPDF(k, k).toFixed(4)}`);
    console.log('');
});

## 4.4 中心極限定理

母集団がどんな分布でも、標本平均の分布は正規分布に近づく

In [ ]:
// 中心極限定理のシミュレーション
console.log('=== 中心極限定理のシミュレーション ===');
console.log('');

// 一様分布からサンプリング（非正規分布）
function uniformRandom(min, max) {
    return min + Math.random() * (max - min);
}

// サンプル平均を多数計算
function simulateSampleMeans(sampleSize, numSimulations) {
    const means = [];
    for (let i = 0; i < numSimulations; i++) {
        const sample = [];
        for (let j = 0; j < sampleSize; j++) {
            sample.push(uniformRandom(0, 10));  // 0-10の一様分布
        }
        means.push(ss.mean(sample));
    }
    return means;
}

// 元の分布（一様分布[0,10]）
console.log('元の分布: 一様分布 U(0, 10)');
console.log('期待値: 5, 分散: 8.33');
console.log('');

// サンプルサイズを変えてシミュレーション
const sampleSizes = [1, 5, 30];
const numSim = 1000;

sampleSizes.forEach(n => {
    const means = simulateSampleMeans(n, numSim);
    console.log(`サンプルサイズ n=${n}:`);
    console.log(`  標本平均の平均: ${ss.mean(means).toFixed(4)} (理論値: 5)`);
    console.log(`  標本平均の標準偏差: ${ss.standardDeviation(means).toFixed(4)} (理論値: ${(Math.sqrt(8.33/n)).toFixed(4)})`);
    console.log(`  歪度: ${ss.sampleSkewness(means).toFixed(4)} (正規分布なら0に近い)`);
    console.log('');
});

console.log('→ サンプルサイズが大きくなるほど正規分布に近づく');

## 4.5 simple-statisticsの確率関数

In [ ]:
console.log('=== simple-statistics の確率関数 ===');
console.log('');

// 累積正規分布
console.log('【累積標準正規分布】');
console.log('P(Z ≤ 1.96) =', ss.cumulativeStdNormalProbability(1.96).toFixed(4));
console.log('');

// パーセント点（分位点）
console.log('【標準正規分布のパーセント点】');
console.log('z₀.₀₂₅ (下側2.5%点) =', ss.probit(0.025).toFixed(4));
console.log('z₀.₉₇₅ (下側97.5%点) =', ss.probit(0.975).toFixed(4));
console.log('');

// エラー関数
console.log('【エラー関数】');
console.log('erf(1) =', ss.errorFunction(1).toFixed(4));
console.log('');

// ベルヌーイ分布
console.log('【ベルヌーイ分布】');
console.log('p=0.3 でのサンプル:', [1,2,3,4,5].map(() => ss.bernoulliDistribution(0.3)));

## 4.6 実践例: 信頼区間の計算

In [ ]:
// 95%信頼区間の計算
const sampleData = [23, 25, 27, 29, 30, 31, 33, 35, 37, 40];

console.log('=== 95%信頼区間の計算 ===');
console.log('サンプルデータ:', sampleData);
console.log('');

const sampleMean = ss.mean(sampleData);
const sampleStd = ss.sampleStandardDeviation(sampleData);
const sampleN = sampleData.length;

console.log('標本平均:', sampleMean);
console.log('標本標準偏差:', sampleStd.toFixed(4));
console.log('サンプルサイズ:', sampleN);
console.log('');

// 標準誤差
const standardError = sampleStd / Math.sqrt(sampleN);
console.log('標準誤差:', standardError.toFixed(4));

// 95%信頼区間（正規分布近似）
const z95 = 1.96;  // 95%信頼区間のz値
const marginOfError = z95 * standardError;

const lowerBound = sampleMean - marginOfError;
const upperBound = sampleMean + marginOfError;

console.log('');
console.log('95%信頼区間:');
console.log(`  ${lowerBound.toFixed(2)} ≤ μ ≤ ${upperBound.toFixed(2)}`);
console.log('');
console.log('解釈: 母平均μは95%の確率でこの区間に含まれる');

## 4.7 練習問題

### 練習1: 二項分布

合格率60%の試験を10人が受験した場合、7人以上合格する確率を求めてください。

In [ ]:
// ここに回答を書いてください



### 練習2: 正規分布

身長が平均170cm、標準偏差6cmの正規分布に従うとき、175cm以上の人の割合を求めてください。

In [ ]:
// ここに回答を書いてください



## まとめ

この章では以下を学びました：

1. **確率の基本**: 条件付き確率、ベイズの定理、組み合わせ・順列
2. **離散確率分布**: 二項分布、ポアソン分布
3. **連続確率分布**: 正規分布、t分布、カイ二乗分布
4. **中心極限定理**: 標本平均の分布は正規分布に近づく
5. **信頼区間**: 母平均の区間推定

次章では、回帰分析について学びます。